In [1]:
import pandas
import json
from os import listdir
from os.path import isfile, join

In [2]:
data_dir = '../dataset/raw/JupyterNotebook'

In [3]:
def get_files(dir_path):
    return [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [4]:
def read_json_file(file_path):
    try:
        return json.load(open(f))
    except json.decoder.JSONDecodeError:
        return None

In [5]:
data = []
'''
data = [
    [{}, {},..., {}], # issue
    [{}, {},..., {}]
]
'''
for f in get_files(data_dir):
    issue = read_json_file(f)
    if issue: # Some files are empty. why tho.
        data.append(read_json_file(f))

In [6]:
data[0]

[{'url': 'https://api.github.com/repos/jupyter/notebook/issues/comments/824259486',
  'html_url': 'https://github.com/jupyter/notebook/issues/6040#issuecomment-824259486',
  'issue_url': 'https://api.github.com/repos/jupyter/notebook/issues/6040',
  'id': 824259486,
  'node_id': 'MDEyOklzc3VlQ29tbWVudDgyNDI1OTQ4Ng==',
  'user': {'login': 'kevin-bates',
   'id': 22599560,
   'node_id': 'MDQ6VXNlcjIyNTk5NTYw',
   'avatar_url': 'https://avatars.githubusercontent.com/u/22599560?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/kevin-bates',
   'html_url': 'https://github.com/kevin-bates',
   'followers_url': 'https://api.github.com/users/kevin-bates/followers',
   'following_url': 'https://api.github.com/users/kevin-bates/following{/other_user}',
   'gists_url': 'https://api.github.com/users/kevin-bates/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/kevin-bates/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/kevin-bates/su

In [7]:
issue_count = 0
data_required = []
for issue in data:
    for comment in issue:
        comment_ind = {}
        comment_ind['issue_count'] = issue_count # to group them to belong to same issue
        comment_ind['id'] = comment['id']
        comment_ind['created_at'] = comment['created_at']
        comment_ind['updated_at'] = comment['updated_at']
        comment_ind['author_association'] = comment['author_association']
        comment_ind['body'] = comment['body']
        comment_ind['user_login'] = comment['user']['login']
        comment_ind['user_type'] = comment['user']['type']
        comment_ind['user_site_admin'] = comment['user']['site_admin']
        data_required.append(comment_ind)
    issue_count += 1

In [8]:
data_df = pandas.DataFrame(data_required)

In [9]:
data_df

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
0,0,824259486,2021-04-21T18:13:22Z,2021-04-21T18:13:22Z,MEMBER,This is a known issue in [jupyter_client](http...,kevin-bates,User,False
1,1,325328641,2017-08-28T11:29:17Z,2017-08-28T11:29:17Z,MEMBER,Thanks - actually heading cells are no longer ...,takluyver,User,False
2,1,325383558,2017-08-28T15:17:48Z,2017-08-28T15:17:48Z,CONTRIBUTOR,Alright. Fixing it now...,syedtshah,User,False
3,1,325409403,2017-08-28T16:51:12Z,2017-08-28T16:51:12Z,MEMBER,Thanks :-),takluyver,User,False
4,2,175818892,2016-01-27T19:44:25Z,2016-01-27T19:44:25Z,MEMBER,This is an iPython bug from what I can tell. I...,jakirkham,User,False
...,...,...,...,...,...,...,...,...,...
30729,4748,526858512,2019-08-31T19:17:41Z,2019-08-31T19:17:41Z,CONTRIBUTOR,@kevin-bates since nobody has reviewed this ye...,benthayer,User,False
30730,4748,526862244,2019-08-31T20:14:56Z,2019-08-31T20:14:56Z,MEMBER,Hi Ben - sorry for the frustration. It certai...,kevin-bates,User,False
30731,4748,526864628,2019-08-31T20:55:56Z,2019-08-31T20:55:56Z,CONTRIBUTOR,@marijnh I see you've been active on the CodeM...,benthayer,User,False
30732,4748,527335475,2019-09-03T07:14:14Z,2019-09-03T07:14:14Z,NONE,"@bthayer2365 Hi. Sorry, too busy to start revi...",marijnh,User,False


In [10]:
data_df.user_site_admin.unique()

array([False,  True])

In [11]:
data_df.user_type.unique()

array(['User', 'Bot'], dtype=object)

In [12]:
data_df.groupby('user_type')['id'].nunique() # 67 bot messages. Need to be cleaned?

user_type
Bot        67
User    30667
Name: id, dtype: int64

In [13]:
data_df.groupby('user_site_admin')['id'].nunique() # 10 Admin messages. What are they?

user_site_admin
False    30724
True        10
Name: id, dtype: int64

In [14]:
data_df[data_df['user_site_admin'] == True]

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
8458,1246,380271464,2018-11-13T15:10:18Z,2021-04-04T00:21:31Z,NONE,Observed behavior: Users periodically open not...,glortho,User,True
9856,1246,252996107,2017-08-25T19:24:10Z,2021-04-04T00:22:43Z,CONTRIBUTOR,Addresses issue #1749 \r\n\r\nHad to add some ...,itsJiaqi,User,True
10078,1246,236846207,2017-06-19T10:27:31Z,2021-04-06T00:16:43Z,NONE,Converting a cell to markdown (via the drop do...,jhelie,User,True
16475,2019,439063661,2018-11-15T14:44:07Z,2018-11-15T19:52:41Z,NONE,We've confirmed that this does appear to happe...,glortho,User,True
16476,2019,439075797,2018-11-15T15:17:42Z,2018-11-15T15:18:09Z,NONE,I could be off-base but it seems like we'd eit...,glortho,User,True
16478,2019,439094745,2018-11-15T16:09:31Z,2018-11-15T16:09:31Z,NONE,Thank you kindly for your attention to this @k...,glortho,User,True
16481,2019,439260107,2018-11-16T02:24:40Z,2018-11-16T02:24:40Z,NONE,"Awesome Kevin!\n\nOn Thu, Nov 15, 2018 at 20:0...",glortho,User,True
20009,2653,309690427,2017-06-20T08:59:40Z,2017-06-20T08:59:40Z,NONE,Closing this as the issue seems to have disapp...,jhelie,User,True
20011,2653,310130539,2017-06-21T16:17:35Z,2017-06-21T16:17:35Z,NONE,@mpacer I'm on 5.0.0 and can't readily downgra...,jhelie,User,True
20436,2746,420352074,2018-09-11T17:21:28Z,2018-09-11T17:21:28Z,NONE,We get this error periodically when first navi...,glortho,User,True


In [15]:
data_df.author_association.unique()

array(['MEMBER', 'CONTRIBUTOR', 'NONE'], dtype=object)

In [16]:
data_df.groupby('author_association')['id'].nunique()

author_association
CONTRIBUTOR     5312
MEMBER         12820
NONE           12602
Name: id, dtype: int64

In [17]:
# sorted(data_df.groupby(['issue_count']).size(), reverse=True)[:30]

In [18]:
description = data_df.groupby(['issue_count']).describe()

In [19]:
description[description['id']['count'] > 30] # Issue number 1246 has a huge discussion. 

id                                                        \
              count          mean           std         min           25%   
issue_count                                                                 
1246         6046.0  3.185080e+08  1.911123e+08  67841415.0  1.584407e+08   

                                                     
                     50%           75%          max  
issue_count                                          
1246         272312823.5  4.286993e+08  891011358.0

In [20]:
long_issue = data_df[data_df["issue_count"] == 1246] # 6046 messages in the conversation

In [21]:
long_issue

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
6615,1246,891011358,2021-05-13T12:49:10Z,2021-05-13T13:23:17Z,MEMBER,Binding sockets hooks them up to the running e...,minrk,User,False
6616,1246,890836693,2021-05-13T08:16:44Z,2021-05-13T08:17:02Z,NONE,**Describe the bug**\r\n\r\nOn French AZERTY k...,bolinocroustibat,User,False
6617,1246,890380684,2021-05-12T18:39:06Z,2021-05-13T13:25:12Z,MEMBER,Automated Changelog Entry for 6.4.0 on master\...,blink1073,User,False
6618,1246,890350101,2021-05-12T18:00:19Z,2021-05-12T18:34:21Z,MEMBER,Automated Changelog Entry for 6.4.0rc0 on mast...,blink1073,User,False
6619,1246,889607938,2021-05-12T04:43:44Z,2021-05-12T23:57:47Z,NONE,\r\n**Describe the bug**\r\nFailed to start th...,shreyansh-vaghela,User,False
...,...,...,...,...,...,...,...,...,...
12656,1246,67841415,2015-04-11T22:47:55Z,2021-04-20T00:17:16Z,MEMBER,None,minrk,User,False
12657,1246,127801152,2015-04-01T16:04:39Z,2021-04-25T00:17:24Z,MEMBER,@ellisonbg opened ipython/ipython#8219\n\nIn p...,Carreau,User,False
12658,1246,110449174,2015-03-24T00:09:35Z,2021-05-02T00:11:37Z,MEMBER,@takluyver opened ipython/ipython#8130\n\nSpot...,minrk,User,False
12659,1246,127459636,2015-03-17T16:14:15Z,2021-04-07T00:16:12Z,MEMBER,@neilzimmerman opened ipython/ipython#8069\n\n...,minrk,User,False


In [22]:
len(long_issue.user_login.unique()) # There are 3182 unique users

3182

Turns out 1296 is probably a concatenated file and it doesn't really have 6046 comments. Oops!

In [23]:
description[description['id']['count'] == 30]

id                                                         \
            count          mean           std          min           25%   
issue_count                                                                
15           30.0  1.913397e+08  1.842859e+04  191311153.0  1.913251e+08   
32           30.0  1.850178e+08  6.706308e+06  165924249.0  1.825982e+08   
60           30.0  3.532083e+08  2.485710e+06  350268218.0  3.502753e+08   
84           30.0  1.993725e+08  2.786559e+07  145981594.0  1.827063e+08   
102          30.0  4.218725e+08  7.007781e+07  293482313.0  3.700471e+08   
...           ...           ...           ...          ...           ...   
4516         30.0  5.359424e+08  2.360125e+07  511135144.0  5.165515e+08   
4539         30.0  1.425471e+08  8.438704e+05  141786410.0  1.418157e+08   
4572         30.0  2.206148e+08  7.204861e+07  137988905.0  1.379932e+08   
4653         30.0  1.345365e+08  4.218945e+06  130901064.0  1.312814e+08   
4746         30.0  5.267365e+08  6.377760e+07  376894001.0  5.135417e+08   

                                                     
                     50%           75%          max  
issue_count                                          
15           191344528.5  1.913578e+08  191364112.0  
32           188392738.0  1.887231e+08  189529710.0  
60           353409948.0  3.556323e+08  355672998.0  
84           202282613.5  2.242672e+08  234600580.0  
102          388532647.5  4.769689e+08  565381574.0  
...                  ...           ...          ...  
4516         519132566.0  5.626268e+08  582960658.0  
4539         142225026.0  1.433147e+08  144157518.0  
4572         229639200.5  2.954265e+08  300482476.0  
4653         132451460.5  1.350448e+08  143398342.0  
4746         557340628.5  5.641277e+08  603913658.0  

[107 rows x 8 columns]

In [24]:
data[15] # https://github.com/jupyter/notebook/pull/1160

[{'url': 'https://api.github.com/repos/jupyter/notebook/issues/comments/191311153',
  'html_url': 'https://github.com/jupyter/notebook/pull/1160#issuecomment-191311153',
  'issue_url': 'https://api.github.com/repos/jupyter/notebook/issues/1160',
  'id': 191311153,
  'node_id': 'MDEyOklzc3VlQ29tbWVudDE5MTMxMTE1Mw==',
  'user': {'login': 'dwillmer',
   'id': 2343358,
   'node_id': 'MDQ6VXNlcjIzNDMzNTg=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/2343358?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/dwillmer',
   'html_url': 'https://github.com/dwillmer',
   'followers_url': 'https://api.github.com/users/dwillmer/followers',
   'following_url': 'https://api.github.com/users/dwillmer/following{/other_user}',
   'gists_url': 'https://api.github.com/users/dwillmer/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/dwillmer/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/dwillmer/subscriptions',
   'organizati

This data seems good to me because
- It contains a mixture of explicit reply_to relations and non explicit ones, also there are explicit mentions
- Discussion is technical as well as non technical + debate
- Closed (an outcome was reached)

In [25]:
long_issue = data_df[data_df["issue_count"] == 15]

In [27]:
long_issue

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
50,15,191311153,2016-03-02T16:21:12Z,2016-03-02T16:21:12Z,MEMBER,"i appreciate your point, but i really think th...",dwillmer,User,False
51,15,191311386,2016-03-02T16:21:56Z,2016-03-02T16:21:56Z,MEMBER,"I'm comfortable either way, since it's on mast...",jasongrout,User,False
52,15,191312128,2016-03-02T16:23:38Z,2016-03-02T16:23:38Z,MEMBER,The other factor in the decision for me is how...,jasongrout,User,False
53,15,191312539,2016-03-02T16:24:31Z,2016-03-02T16:24:31Z,MEMBER,But to @dwillmer's point - let's have a meetin...,jasongrout,User,False
54,15,191321329,2016-03-02T16:48:12Z,2016-03-02T16:48:12Z,MEMBER,We definitely aren't close to releasing 5.0. W...,minrk,User,False
55,15,191323744,2016-03-02T16:54:28Z,2016-03-02T16:54:28Z,CONTRIBUTOR,"It can currently be used to edit text, run ter...",sccolbert,User,False
56,15,191324329,2016-03-02T16:56:29Z,2016-03-02T16:57:19Z,CONTRIBUTOR,"There are people running on master, complainin...",sccolbert,User,False
57,15,191324873,2016-03-02T16:58:14Z,2016-03-02T16:58:14Z,MEMBER,"No, there are people complaining that an extre...",minrk,User,False
58,15,191325585,2016-03-02T16:59:50Z,2016-03-02T17:00:02Z,CONTRIBUTOR,These are developers running from master thoug...,sccolbert,User,False
59,15,191325892,2016-03-02T17:00:33Z,2016-03-02T17:00:33Z,MEMBER,I don't see why delaying the introduction of a...,minrk,User,False


In [26]:
len(long_issue.user_login.unique()) # There are 9 unique users

9

In [31]:
from datetime import datetime
dt_list = [datetime.fromisoformat(t[:-1]) for t in long_issue.created_at]

In [41]:
d

datetime.timedelta(seconds=2, microseconds=258889)